# 한국어 텍스트 감정 분석 - WandB Sweep 하이퍼파라미터 튜닝

이 노트북은 WandB Sweep을 사용하여 최적의 하이퍼파라미터를 찾습니다.

## 주요 기능
- WandB Sweep을 통한 자동 하이퍼파라미터 튜닝
- Bayesian Optimization 방법 사용
- 튜닝 대상: learning_rate, batch_size, label_smoothing_factor, weight_decay, num_epochs, augment_ratio, newly_gen_ratio
- 데이터셋 비율 조정: original(1.0) : augment(0.5-2.0) : newly_generated(0.5-2.0)
- 최적화 목표: 검증 정확도 최대화
- 10-20회 실행으로 효율적인 탐색


In [1]:
# Library Import
import os
import math
import warnings
from collections import Counter
import platform
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR
from transformers import get_linear_schedule_with_warmup
from torch.cuda.amp import autocast, GradScaler
from tqdm.auto import tqdm
import wandb

# Transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    set_seed
)

# PEFT for LoRA
from peft import get_peft_model, LoraConfig, TaskType

# Sklearn
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 경고 메시지 필터링
warnings.filterwarnings("ignore")

print("✅ 라이브러리 임포트 완료")


/data/ephemeral/home/py310/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/data/ephemeral/home/py310/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotat

✅ 라이브러리 임포트 완료


In [2]:
# 환경 설정
from dotenv import load_dotenv
import os

# 이 함수가 .env 파일을 읽어서 환경 변수로 로드합니다.
load_dotenv()

RANDOM_STATE = 42
set_seed(RANDOM_STATE)

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"디바이스: {device}")

if torch.cuda.is_available():
    print(f"GPU 개수: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("⚠️  CUDA 사용 불가 - CPU로 훈련 진행")


디바이스: cuda
GPU 개수: 1
   GPU 0: Tesla V100-SXM2-32GB


In [3]:
import datetime
now = datetime.datetime.now()
TIMESTAMP = now.strftime("%Y-%m-%d_%H-%M-%S")

# 고정 설정
PROJECT_NAME = "[domain_project]_Sweep_Experiments"
MODEL_NAME = "kykim_bert-base"
RUN_NAME = f"{MODEL_NAME}_sweep_hyperparameter_tuning"

os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [4]:
# WandB Sweep 설정
sweep_config = {
    'method': 'bayes',  # Bayesian optimization
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'learning_rate': {
            'distribution': 'log_uniform_values',
            'min': 1e-5,
            'max': 5e-4
        },
        'batch_size': {
            'values': [512]
        },
        'label_smoothing_factor': {
            'distribution': 'uniform',
            'min': 0.05,
            'max': 0.15
        },
        'weight_decay': {
            'distribution': 'log_uniform_values',
            'min': 0.005,
            'max': 0.1
        },
        'num_epochs': {
            'values': [5]
        },
        'augment_ratio': {
            'values': [1.0, 2.0, 3.0]
        },
        'newly_gen_ratio': {
            'values': [0.0]
        }
    },
    'early_terminate': {
        'type': 'hyperband',
        'min_iter': 2,
        'eta': 2
    }
}

print("✅ WandB Sweep 설정 완료")
print(f"튜닝 대상: {list(sweep_config['parameters'].keys())}")
print(f"최적화 목표: {sweep_config['metric']['name']} ({sweep_config['metric']['goal']})")
print(f"데이터셋 비율 범위:")
print(f"  - augment_ratio: {sweep_config['parameters']['augment_ratio']['values']}")
print(f"  - newly_gen_ratio: {sweep_config['parameters']['newly_gen_ratio']['values']}")
print(f"  - original은 항상 1.0 (기준)")


✅ WandB Sweep 설정 완료
튜닝 대상: ['learning_rate', 'batch_size', 'label_smoothing_factor', 'weight_decay', 'num_epochs', 'augment_ratio', 'newly_gen_ratio']
최적화 목표: val_accuracy (maximize)
데이터셋 비율 범위:
  - augment_ratio: [1.0, 2.0, 3.0]
  - newly_gen_ratio: [0.0]
  - original은 항상 1.0 (기준)


In [5]:
# 고정 하이퍼파라미터 설정
class FixedConfig:
    # 모델 설정
    model_name = MODEL_NAME
    base_models_path = "/data/ephemeral/home/code/basemodels/"
    local_model_path = base_models_path + MODEL_NAME

    is_DAPT = True
    DAPT_model_path = "./best_unsupervised_model/TAPT_kykim_bert-kor-base_augX3_best_unsupervised_model_1028"
    
    save_model_path = "./DAPT_fine/"+ RUN_NAME

    num_classes = 4
    max_length = 128

    # 데이터 설정
    train_data_path = "/data/ephemeral/home/code/data/train_final_augX3_newly_gen_added.csv"
    val_data_path = "/data/ephemeral/home/code/data/val_final.csv"
    
    # 고정 훈련 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    warmup_steps = 500
    
    # 기타 설정
    gradient_accumulation_steps = 1
    max_grad_norm = 1.0
    early_stopping_patience = 5
    save_best_model = False  # Sweep에서는 개별 모델 저장 안함
    
    # WandB 설정
    use_wandb = True
    project_name = PROJECT_NAME

    # Loss 설정
    loss_function = "CrossEntropy"
    use_label_smoothing = True  # Sweep에서 튜닝됨

    use_lora = False

fixed_config = FixedConfig()
print("✅ 고정 설정 완료")


✅ 고정 설정 완료


In [6]:
# 데이터셋 비율에 따른 샘플링 함수
def sample_dataset_by_ratio(df, augment_ratio=1.0, newly_gen_ratio=1.0, random_state=42):
    """
    데이터셋을 비율에 따라 샘플링하는 함수
    
    Args:
        df: 전체 데이터프레임
        augment_ratio: augment 데이터 비율 (original 대비)
        newly_gen_ratio: newly_generated 데이터 비율 (original 대비)
        random_state: 랜덤 시드
    
    Returns:
        sampled_df: 샘플링된 데이터프레임
    """
    np.random.seed(random_state)
    
    # 각 타입별 데이터 분리
    original_data = df[df['type'] == 'original'].copy()
    augment_data = df[df['type'] == 'augment'].copy()
    newly_gen_data = df[df['type'] == 'newly_generated'].copy()
    
    print(f"원본 데이터 분포:")
    print(f"  Original: {len(original_data):,}개")
    print(f"  Augment: {len(augment_data):,}개")
    print(f"  Newly_generated: {len(newly_gen_data):,}개")
    
    # Original 데이터는 항상 100% 사용
    sampled_original = original_data.copy()
    
    # Augment 데이터 샘플링
    if augment_ratio > 0 and len(augment_data) > 0:
        # original 대비 비율로 샘플링
        target_augment_size = int(len(original_data) * augment_ratio)
        if target_augment_size > len(augment_data):
            # 요청된 크기가 전체보다 크면 전체 사용
            sampled_augment = augment_data.copy()
        else:
            # 비율에 맞게 샘플링
            sampled_augment = augment_data.sample(n=target_augment_size, random_state=random_state)
    else:
        sampled_augment = pd.DataFrame(columns=df.columns)
    
    # Newly_generated 데이터 샘플링
    if newly_gen_ratio > 0 and len(newly_gen_data) > 0:
        # original 대비 비율로 샘플링
        target_newly_gen_size = int(len(original_data) * newly_gen_ratio)
        if target_newly_gen_size > len(newly_gen_data):
            # 요청된 크기가 전체보다 크면 전체 사용
            sampled_newly_gen = newly_gen_data.copy()
        else:
            # 비율에 맞게 샘플링
            sampled_newly_gen = newly_gen_data.sample(n=target_newly_gen_size, random_state=random_state)
    else:
        sampled_newly_gen = pd.DataFrame(columns=df.columns)
    
    # 샘플링된 데이터 합치기
    sampled_df = pd.concat([sampled_original, sampled_augment, sampled_newly_gen], ignore_index=True)
    
    print(f"\\n샘플링된 데이터 분포:")
    print(f"  Original: {len(sampled_original):,}개 (비율: 1.0)")
    print(f"  Augment: {len(sampled_augment):,}개 (비율: {augment_ratio:.1f})")
    print(f"  Newly_generated: {len(sampled_newly_gen):,}개 (비율: {newly_gen_ratio:.1f})")
    print(f"  총 데이터: {len(sampled_df):,}개")
    
    return sampled_df

print("✅ 데이터셋 샘플링 함수 정의 완료")


✅ 데이터셋 샘플링 함수 정의 완료


In [7]:
# 전체 데이터 로드 (비율 샘플링을 위해)
print("전체 데이터 로드 중...")

# 원본 데이터 로드 (preprocessing_final.ipynb에서 생성된 데이터)
full_df = pd.read_csv(fixed_config.train_data_path)
print(f"전체 데이터: {len(full_df):,}개")

# 검증 데이터 로드 (original만 사용)
val_df = pd.read_csv(fixed_config.val_data_path)
print(f"검증 데이터: {len(val_df):,}개")

print("✅ 데이터 로드 완료")


전체 데이터 로드 중...
전체 데이터: 627,637개
검증 데이터: 6,823개
✅ 데이터 로드 완료


In [8]:
# 데이터셋 클래스 정의
class ReviewDataset(Dataset):
    """
    리뷰 텍스트 데이셋 클래스
    - BERT 모델 훈련/추론을 위한 PyTorch Dataset 구현
    - 텍스트 토크나이징 및 텐서 변환 처리
    """

    def __init__(self, texts, labels, tokenizer, max_length):
        """
        데이터셋 초기화
        """
        self.texts, self.labels, self.tokenizer, self.max_length = (
            texts,
            labels,
            tokenizer,
            max_length,
        )

    def __len__(self):
        """데이터셋 크기 반환"""
        return len(self.texts)

    def __getitem__(self, idx):
        """
        특정 인덱스의 데이터 아이템 반환
        """
        # 텍스트 토크나이징 및 패딩
        encoding = self.tokenizer(
            str(self.texts.iloc[idx]),
            truncation=True,  # 최대 길이 초과시 자르기
            padding="max_length",  # 최대 길이까지 패딩
            max_length=self.max_length,
            return_tensors="pt",  # PyTorch 텐서로 반환
        )

        # 기본 아이템 구성 (input_ids, attention_mask)
        item = {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
        }

        # labels가 None이 아닌 경우에만 추가 (train/valid용)
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels.iloc[idx], dtype=torch.long)

        return item

print("✅ 데이터셋 클래스 정의 완료")


✅ 데이터셋 클래스 정의 완료


In [9]:
# 평가 메트릭 함수
def compute_metrics(predictions, labels):
    """
    모델 평가 메트릭 계산 함수
    """
    # 예측값에서 가장 높은 확률의 클래스 선택
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    
    return {
        "accuracy": accuracy,
        "f1": f1,
    }

print("✅ 평가 메트릭 함수 정의 완료")


✅ 평가 메트릭 함수 정의 완료


In [10]:
def model_load_from_local(local_model_path : str = None, model_name : str = None):
    # 1. 토크나이저 로드
    # 절대 경로를 사용하여 로컬 모델 로드
    print("로컬 경로에서 토크나이저 로딩 중...")

    # 경로가 존재하는지 확인
    if not os.path.exists(local_model_path):
        print(f"❌ 경로가 존재하지 않습니다: {local_model_path}")
        return None, None
    else:
        print(f"✅ 경로 확인됨: {local_model_path}")

    tokenizer = AutoTokenizer.from_pretrained(local_model_path)

    # 2. 모델 로드
    # 마찬가지로 로컬 경로(local_model_path)를 사용
    print("로컬 경로에서 모델 로딩 중...")
    model = AutoModelForSequenceClassification.from_pretrained(
        local_model_path,
        num_labels=4,
        ignore_mismatched_sizes=True
    )

    print("✅ 로컬 스냅샷에서 모델과 토크나이저 로딩 성공!")

    return model, tokenizer


In [11]:
import torch.nn.functional as F

def label_smoothing_cross_entropy(predictions, targets, smoothing=0.1):
    """
    predictions: 모델 출력 (logits) [batch_size, num_classes]
    targets: 정답 라벨 (정수) [batch_size]
    smoothing: smoothing factor
    """
    num_classes = predictions.size(-1)
    
    # 하드 라벨을 원-핫으로 변환
    true_dist = torch.zeros_like(predictions)
    true_dist.fill_(smoothing / (num_classes - 1))
    true_dist.scatter_(1, targets.unsqueeze(1), 1.0 - smoothing)
    
    # KL divergence = Cross Entropy with smoothed labels
    return F.kl_div(F.log_softmax(predictions, dim=1), true_dist, reduction='batchmean')

print("✅ Label Smoothing 함수 정의 완료")


✅ Label Smoothing 함수 정의 완료


In [12]:
def train_epoch(model, dataloader, optimizer, scheduler, scaler, device, config, criterion_fn):
    model.train()
    total_loss = 0
    predictions = []
    labels = []
    
    progress_bar = tqdm(dataloader, desc="Training")
    
    for step, batch in enumerate(progress_bar):
        # 배치를 디바이스로 이동
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        batch_labels = batch["labels"].to(device)
        
        # 혼합 정밀도 훈련
        if scaler is not None:
            with autocast():
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                )

                logits = outputs.logits
                loss = criterion_fn(logits, batch_labels) / config.gradient_accumulation_steps
            
            scaler.scale(loss).backward()
            
            if (step + 1) % config.gradient_accumulation_steps == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                optimizer.zero_grad()
        else:
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            logits = outputs.logits
            loss = criterion_fn(logits, batch_labels) / config.gradient_accumulation_steps
            
            loss.backward()
            
            if (step + 1) % config.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
        
        total_loss += loss.item() * config.gradient_accumulation_steps
        
        # 예측값 저장
        predictions.extend(logits.detach().cpu().numpy())
        labels.extend(batch_labels.detach().cpu().numpy())
        
        # 진행률 업데이트
        progress_bar.set_postfix({
            'loss': f'{loss.item() * config.gradient_accumulation_steps:.4f}',
            'lr': f'{scheduler.get_last_lr()[0]:.2e}'
        })
    
    # 메트릭 계산
    metrics = compute_metrics(predictions, labels)
    metrics['loss'] = total_loss / len(dataloader)
    
    return metrics


In [13]:
# 검증 함수
def validate_epoch(model, dataloader, device, criterion_fn):
    """한 에포크 검증"""
    model.eval()
    total_loss = 0
    predictions = []
    labels = []
    
    progress_bar = tqdm(dataloader, desc="Validation")
    
    with torch.no_grad():
        for batch in progress_bar:
            # 배치를 디바이스로 이동
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            batch_labels = batch["labels"].to(device)
            
            # 순전파
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            
            logits = outputs.logits
            loss = criterion_fn(logits, batch_labels)
            
            total_loss += loss.item()
            
            # 예측값 저장
            predictions.extend(logits.detach().cpu().numpy())
            labels.extend(batch_labels.detach().cpu().numpy())
            
            # 진행률 업데이트
            progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    # 메트릭 계산
    metrics = compute_metrics(predictions, labels)
    metrics['loss'] = total_loss / len(dataloader)
    
    return metrics

print("✅ 검증 함수 정의 완료")


✅ 검증 함수 정의 완료


In [14]:
# 메인 훈련 함수 (Sweep용)
def train():
    # WandB 초기화
    run = wandb.init()
    
    # Sweep에서 받은 하이퍼파라미터
    config = wandb.config
    
    print(f"\n{'='*50}")
    print(f"Sweep Run 시작")
    print(f"learning_rate: {config.learning_rate:.2e}")
    print(f"batch_size: {config.batch_size}")
    print(f"label_smoothing_factor: {config.label_smoothing_factor:.3f}")
    print(f"weight_decay: {config.weight_decay:.3f}")
    print(f"num_epochs: {config.num_epochs}")
    print(f"augment_ratio: {config.augment_ratio:.1f}")
    print(f"newly_gen_ratio: {config.newly_gen_ratio:.1f}")
    print(f"{'='*50}")
    
    try:
        # 모델 및 토크나이저 로드
        if fixed_config.is_DAPT:
            model, tokenizer = model_load_from_local(fixed_config.DAPT_model_path, fixed_config.model_name)
        else:
            model, tokenizer = model_load_from_local(fixed_config.local_model_path, fixed_config.model_name)
        
        if model is None or tokenizer is None:
            print("❌ 모델 로드 실패")
            return
        
        model = model.to(fixed_config.device)
        
        # 비율에 따라 훈련 데이터 샘플링
        print("\\n데이터셋 비율에 따른 샘플링 중...")
        train_df_sampled = sample_dataset_by_ratio(
            full_df, 
            augment_ratio=config.augment_ratio, 
            newly_gen_ratio=config.newly_gen_ratio,
            random_state=RANDOM_STATE
        )
        
        # 데이터셋 및 데이터로더 생성
        train_dataset = ReviewDataset(
            train_df_sampled["review"],
            train_df_sampled["label"],
            tokenizer,
            fixed_config.max_length,
        )

        val_dataset = ReviewDataset(
            val_df["review"],
            val_df["label"],
            tokenizer,
            fixed_config.max_length,
        )

        train_dataloader = DataLoader(
            train_dataset,
            batch_size=config.batch_size,
            shuffle=True,
            num_workers=0,
            pin_memory=True if torch.cuda.is_available() else False
        )

        val_dataloader = DataLoader(
            val_dataset,
            batch_size=config.batch_size,
            shuffle=False,
            num_workers=0,
            pin_memory=True if torch.cuda.is_available() else False
        )
        
        # 손실 함수 설정
        def criterion_fn(predictions, targets):
            return label_smoothing_cross_entropy(predictions, targets, config.label_smoothing_factor)
        
        # 옵티마이저 설정
        optimizer = AdamW(
            model.parameters(),
            lr=config.learning_rate,
            weight_decay=config.weight_decay
        )

        # 전체 훈련 스텝 계산
        total_steps = len(train_dataloader) * config.num_epochs // fixed_config.gradient_accumulation_steps

        # 스케줄러 설정 (warmup + linear decay)
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=fixed_config.warmup_steps,
            num_training_steps=total_steps
        )

        # 혼합 정밀도 스케일러
        scaler = GradScaler() if torch.cuda.is_available() else None
        
        # 훈련 루프
        best_val_accuracy = 0
        
        for epoch in range(config.num_epochs):
            # 훈련
            train_metrics = train_epoch(
                model, train_dataloader, optimizer, scheduler, scaler, 
                fixed_config.device, fixed_config, criterion_fn
            )
            
            # 검증
            val_metrics = validate_epoch(model, val_dataloader, fixed_config.device, criterion_fn)
            
            # WandB 로깅
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": train_metrics['loss'],
                "train_accuracy": train_metrics['accuracy'],
                "train_f1": train_metrics['f1'],
                "val_loss": val_metrics['loss'],
                "val_accuracy": val_metrics['accuracy'],
                "val_f1": val_metrics['f1'],
                "learning_rate": scheduler.get_last_lr()[0]
            })
            
            # 최고 성능 추적
            if val_metrics['accuracy'] > best_val_accuracy:
                best_val_accuracy = val_metrics['accuracy']
        
        print(f"✅ 훈련 완료! 최고 검증 정확도: {best_val_accuracy:.4f}")
        
        # 최종 메트릭 로깅
        wandb.log({"best_val_accuracy": best_val_accuracy})
        
    except Exception as e:
        print(f"❌ 훈련 중 오류 발생: {str(e)}")
        wandb.log({"error": str(e)})
    finally:
        wandb.finish()

print("✅ 훈련 함수 정의 완료")


✅ 훈련 함수 정의 완료


In [15]:
# Sweep 실행
print("WandB Sweep 시작...")
print(f"프로젝트: {PROJECT_NAME}")
print(f"실행 횟수: 10회")
print(f"최적화 목표: 검증 정확도 최대화")

# Sweep ID 생성
sweep_id = wandb.sweep(
    sweep_config,
    project=PROJECT_NAME
)

print(f"\n✅ Sweep 생성 완료!")
print(f"Sweep ID: {sweep_id}")
print(f"Sweep URL: https://wandb.ai/{wandb.api.default_entity}/{PROJECT_NAME}/sweeps/{sweep_id}")
print("\nSweep 실행을 시작합니다...")


WandB Sweep 시작...
프로젝트: [domain_project]_Sweep_Experiments
실행 횟수: 15회
최적화 목표: 검증 정확도 최대화
Create sweep with ID: awpthshs
Sweep URL: https://wandb.ai/qkdwodus777-/%5Bdomain_project%5D_Sweep_Experiments/sweeps/awpthshs

✅ Sweep 생성 완료!
Sweep ID: awpthshs
Sweep URL: https://wandb.ai/qkdwodus777-/[domain_project]_Sweep_Experiments/sweeps/awpthshs

Sweep 실행을 시작합니다...


In [ ]:
# Sweep Agent 실행
wandb.agent(
    sweep_id,
    function=train,
    count=10,  # 10-20회 중 15회 실행
    project=PROJECT_NAME
)

print("\n🎉 Sweep 완료!")


wandb: Agent Starting Run: l1ynaqtz with config:
wandb: 	augment_ratio: 3
wandb: 	batch_size: 512
wandb: 	label_smoothing_factor: 0.11885811974590456
wandb: 	learning_rate: 5.255921600625229e-05
wandb: 	newly_gen_ratio: 0
wandb: 	num_epochs: 5
wandb: 	weight_decay: 0.07235098370604887
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin



Sweep Run 시작
learning_rate: 5.26e-05
batch_size: 512
label_smoothing_factor: 0.119
weight_decay: 0.072
num_epochs: 5
augment_ratio: 3.0
newly_gen_ratio: 0.0
로컬 경로에서 토크나이저 로딩 중...
✅ 경로 확인됨: ./best_unsupervised_model/TAPT_kykim_bert-kor-base_augX3_best_unsupervised_model_1028
로컬 경로에서 모델 로딩 중...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./best_unsupervised_model/TAPT_kykim_bert-kor-base_augX3_best_unsupervised_model_1028 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ 로컬 스냅샷에서 모델과 토크나이저 로딩 성공!
\n데이터셋 비율에 따른 샘플링 중...
원본 데이터 분포:
  Original: 129,630개
  Augment: 407,336개
  Newly_generated: 90,671개
\n샘플링된 데이터 분포:
  Original: 129,630개 (비율: 1.0)
  Augment: 388,890개 (비율: 3.0)
  Newly_generated: 0개 (비율: 0.0)
  총 데이터: 518,520개


Training:   0%|          | 0/1013 [00:00<?, ?it/s]

...

Training:   0%|          | 0/760 [00:00<?, ?it/s]

Validation:   0%|          | 0/14 [00:00<?, ?it/s]

Training:   0%|          | 0/760 [00:00<?, ?it/s]

Validation:   0%|          | 0/14 [00:00<?, ?it/s]

Training:   0%|          | 0/760 [00:00<?, ?it/s]

## Sweep 결과 분석


In [ ]:
# 최고 성능 파라미터 조회
api = wandb.Api()
sweep = api.sweep(f"{wandb.api.default_entity}/{PROJECT_NAME}/sweeps/{sweep_id}")

# 최고 성능 run 찾기
best_run = None
best_accuracy = 0

for run in sweep.runs:
    if run.summary.get('best_val_accuracy', 0) > best_accuracy:
        best_accuracy = run.summary.get('best_val_accuracy', 0)
        best_run = run

if best_run:
    print("🏆 최고 성능 파라미터:")
    print(f"  검증 정확도: {best_accuracy:.4f}")
    print(f"  learning_rate: {best_run.config['learning_rate']:.2e}")
    print(f"  batch_size: {best_run.config['batch_size']}")
    print(f"  label_smoothing_factor: {best_run.config['label_smoothing_factor']:.3f}")
    print(f"  weight_decay: {best_run.config['weight_decay']:.3f}")
    print(f"  num_epochs: {best_run.config['num_epochs']}")
    print(f"\n  Run URL: {best_run.url}")
else:
    print("❌ 최고 성능 run을 찾을 수 없습니다.")

print(f"\n📊 전체 Sweep 결과: {len(sweep.runs)}개 run 완료")


In [ ]:
# 하이퍼파라미터 중요도 분석
import matplotlib.pyplot as plt
import seaborn as sns

# 모든 run의 결과 수집
results = []
for run in sweep.runs:
    if run.summary.get('best_val_accuracy') is not None:
        results.append({
            'learning_rate': run.config['learning_rate'],
            'batch_size': run.config['batch_size'],
            'label_smoothing_factor': run.config['label_smoothing_factor'],
            'weight_decay': run.config['weight_decay'],
            'num_epochs': run.config['num_epochs'],
            'val_accuracy': run.summary.get('best_val_accuracy', 0)
        })

if results:
    results_df = pd.DataFrame(results)
    
    # 상관관계 분석
    plt.figure(figsize=(15, 10))
    
    # 1. learning_rate vs val_accuracy
    plt.subplot(2, 3, 1)
    plt.scatter(results_df['learning_rate'], results_df['val_accuracy'], alpha=0.6)
    plt.xscale('log')
    plt.xlabel('Learning Rate')
    plt.ylabel('Validation Accuracy')
    plt.title('Learning Rate vs Validation Accuracy')
    
    # 2. batch_size vs val_accuracy
    plt.subplot(2, 3, 2)
    sns.boxplot(data=results_df, x='batch_size', y='val_accuracy')
    plt.xlabel('Batch Size')
    plt.ylabel('Validation Accuracy')
    plt.title('Batch Size vs Validation Accuracy')
    
    # 3. label_smoothing_factor vs val_accuracy
    plt.subplot(2, 3, 3)
    plt.scatter(results_df['label_smoothing_factor'], results_df['val_accuracy'], alpha=0.6)
    plt.xlabel('Label Smoothing Factor')
    plt.ylabel('Validation Accuracy')
    plt.title('Label Smoothing vs Validation Accuracy')
    
    # 4. weight_decay vs val_accuracy
    plt.subplot(2, 3, 4)
    plt.scatter(results_df['weight_decay'], results_df['val_accuracy'], alpha=0.6)
    plt.xscale('log')
    plt.xlabel('Weight Decay')
    plt.ylabel('Validation Accuracy')
    plt.title('Weight Decay vs Validation Accuracy')
    
    # 5. num_epochs vs val_accuracy
    plt.subplot(2, 3, 5)
    sns.boxplot(data=results_df, x='num_epochs', y='val_accuracy')
    plt.xlabel('Number of Epochs')
    plt.ylabel('Validation Accuracy')
    plt.title('Epochs vs Validation Accuracy')
    
    # 6. 빈 공간 (또는 추가 분석)
    plt.subplot(2, 3, 6)
    plt.text(0.5, 0.5, 'Additional\nAnalysis\nSpace', 
             ha='center', va='center', fontsize=12, alpha=0.5)
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # 상관관계 매트릭스
    plt.figure(figsize=(8, 6))
    correlation_matrix = results_df.corr()
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
    plt.title('Hyperparameter Correlation Matrix')
    plt.show()
    
    print("\n📈 하이퍼파라미터 상관관계 분석 완료")
else:
    print("❌ 분석할 결과가 없습니다.")


## 최적 파라미터를 사용한 최종 훈련

위에서 찾은 최적 파라미터를 사용하여 `finetuning_pytorch.ipynb`의 Config 클래스를 다음과 같이 수정하세요:

```python
class Config:
    # ... 기존 설정들 ...
    
    # 최적화된 하이퍼파라미터 (Sweep 결과)
    batch_size = [최적값]
    learning_rate = [최적값]
    weight_decay = [최적값]
    label_smoothing_factor = [최적값]
    num_epochs = [최적값]
    
    # ... 나머지 설정들 ...
```
